<a href="https://colab.research.google.com/github/cbrss/Programacion_concurrente_M1/blob/main/PC_TP_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>